### 预处理
ML, DL工程师很多任务都是预处理

#### 步骤：
1. 分词
2. 建立词表：词语 -> id #embedding
3. label -> id


In [1]:
#matrix -> [|V|, embed_size] #行是词个数
#词语A -> id(5) #词语A对应着matrix的第5行

In [2]:
import sys
import os
import jieba #分词的库：pip install jieba

In [3]:
output_dir = '../../../other_datasets/cnews_output'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [4]:
#输入文件：训练集，测试集，验证集
train_file = '../../../other_datasets/text_classification_data/cnews.train.txt'
val_file = '../../../other_datasets/text_classification_data/cnews.val.txt'
test_file = '../../../other_datasets/text_classification_data/cnews.test.txt'

#输出文件：三个文件的分别的分词结果
seg_train_file = '../../../other_datasets/cnews_output/cnews.train.seg.txt'
seg_val_file = '../../../other_datasets/cnews_output/cnews.val.seg.txt'
seg_test_file = '../../../other_datasets/cnews_output/cnews.test.seg.txt'

#词表文件：词语 到 id的映射
vocab_file = '../../../other_datasets/cnews_output/cnews.vocab.txt'

#label 到 id的映射
category_file = '../../../other_datasets/cnews_output/cnews.category.txt'

In [5]:
#如何使用jieba库
with open(val_file, 'r') as f: #打开val_file文件，并且读，'r'意思是read
    lines = f.readlines()
print(lines[0])

体育	黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛首轮洛杉矶湖人主场迎战新奥尔良黄蜂，此前的比赛中，双方战成2-2平，因此本场比赛对于两支球队来说都非常重要，赛前双方也公布了首发阵容：湖人队：费舍尔、科比、阿泰斯特、加索尔、拜纳姆黄蜂队：保罗、贝里内利、阿里扎、兰德里、奥卡福[新浪NBA官方微博][新浪NBA湖人新闻动态微博][新浪NBA专题][黄蜂vs湖人图文直播室](新浪体育)



In [6]:
#因为我们的.txt存的内容是utf-8格式，我们需要对utf-8接码，成为Unicode： .encode().decode('utf-8')
#再将最后的换行符去掉
#因为label体育 和 content是用一个table键连接的，所以用split('\t')
label, content = lines[0].encode().decode('utf-8').strip('\r\n').split('\t')
print(label)

体育


### 分词

In [7]:
#将content输入jieba库，得到一个iterator
words_iter = jieba.cut(content)

In [8]:
print('/'.join(words_iter)) #jieba 分的还是不错的

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/h6/z6wvxnf17z15xhtgs3swqx080000gn/T/jieba.cache
Loading model cost 0.618 seconds.
Prefix dict has been built successfully.


黄蜂/vs/湖人/首发/：/科比/带伤/战/保罗/ /加索尔/救赎/之战/ /新浪/体育讯/北京/时间/4/月/27/日/，/NBA/季后赛/首轮/洛杉矶/湖人/主场/迎战/新奥尔良/黄蜂/，/此前/的/比赛/中/，/双方/战成/2/-/2/平/，/因此/本场/比赛/对于/两支/球队/来说/都/非常/重要/，/赛前/双方/也/公布/了/首发/阵容/：/湖人队/：/费舍尔/、/科比/、/阿泰斯特/、/加索尔/、/拜纳姆/黄蜂队/：/保罗/、/贝里/内利/、/阿里/扎/、/兰德/里/、/奥卡福/[/新浪/NBA/官方/微博/]/[/新浪/NBA/湖人/新闻动态/微博/]/[/新浪/NBA/专题/]/[/黄蜂/vs/湖人/图文/直播室/]/(/新浪/体育/)


In [9]:
print(content)

黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛首轮洛杉矶湖人主场迎战新奥尔良黄蜂，此前的比赛中，双方战成2-2平，因此本场比赛对于两支球队来说都非常重要，赛前双方也公布了首发阵容：湖人队：费舍尔、科比、阿泰斯特、加索尔、拜纳姆黄蜂队：保罗、贝里内利、阿里扎、兰德里、奥卡福[新浪NBA官方微博][新浪NBA湖人新闻动态微博][新浪NBA专题][黄蜂vs湖人图文直播室](新浪体育)


In [10]:
#生成分词后的文件
def generate_seg_file(input_file, output_seg_file):
    """Segment the sentences in each line in input_file"""
    #这里用比较暴力的方法，因为文件小所以将其全部读入。
    #但是如果遇到大文件，应该是读一行处理一行
    with open(input_file, 'r') as f: #打开input_file文件，并且读，'r'意思是read
        lines = f.readlines()
    with open(output_seg_file, 'w') as f:
        for line in lines:
            label, content = line.encode().decode('utf-8').strip('\r\n').split('\t')
            word_iter = jieba.cut(content)
            #因为jieba会把空格也当成一个词，所以我们要把空格删去：
            word_content = ''
            for word in word_iter:
                word = word.strip(' ') #word里面如果有空格，不管是几个空格，都会被删除。如果word本身是空格，word会变成空字符串
                if word != '':
                    word_content += word + ' ' #我们用空格来分隔词与词，而不是之前用 \ 符号分隔
            out_line = '%s\t%s\n' % (label, word_content.strip(' ')) #也就是label \t word_content \n
            f.write(out_line) #将out_line写入output_seg_file
            #如果运行不了，可能是因为我们转码成了Unicode，如果需要写入文件，还需要将Unicode转回城utf-8
            #f.write(out_line.encode('utf-8'))
            
# generate_seg_file(train_file, seg_train_file)
# generate_seg_file(val_file, seg_val_file)
# generate_seg_file(test_file, seg_test_file)

### 构建词表

In [11]:
def generate_vocab_file(input_seg_file, output_vocab_file):
    #输入是：分词后的文件
    #输出是：词表
    with open(input_seg_file, 'r') as f:
        lines = f.readlines()
        
    #存储词语
    word_dict = {}
    
    #id 和 索引 联系起来
    for line in lines:
        label, content = line.strip('\r\n').encode().decode('utf-8').split('\t')
        
        #计算所有词语的频次
        for word in content.split():#split()默认是将空格拆分
            word_dict.setdefault(word, 0) #setdefault(): 如果没有这个key，就加入并设置val为0。如果有key就略过
            word_dict[word] += 1
        
    #词频的逆排序：频率高的在前面
    #sorted()： 对.item， 其中item的第2个参数是频次（用lambda d : d[1] 表示），逆序排序
    #sorted_word_dict: 一个list，里面有很多元组，每个元组有两个元素：
    #[(word, freq), (), (), ..., ()]
    sorted_word_dict = sorted(word_dict.items(), key = lambda d : d[1], reverse = True)
    
    #将sorted_word_dict输出
    with open(output_vocab_file, 'w') as f:
        #定义一个special词<UNK> : unknown, 意思是训练集上没有出现，可是测试集上出现
        f.write('<UNK>\t1000000\n') #\t 表示table，1000000是一百万的意思
        a = 0
        for item in sorted_word_dict:
            #f.write('a\n')
            f.write('%s\t%d\n' % (item[0], item[1]) )
#             if a < 50:
#                 #f.write('%s\t%d\n' % (item[0].encode('utf-8'), item[1]) )
                
#                 a += 1
            #
            
#生成词表，需要用训练集生成词表
#真实环境中，没有测试集，测试集是线上数据
generate_vocab_file(seg_train_file, vocab_file)

#对于出现次数少的词，梯度只会计算比较少次数，结果不可信

In [16]:
#如何使用jieba库
with open(vocab_file, 'r') as f: #打开val_file文件，并且读，'r'意思是read
    lines = f.readlines()
print(lines[2])
print(lines[6])

的	1390830

了	248160

